In [2]:
import os
# Disable tensorflow warning messages.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
import numpy as np
import tensorflow as tf

## Load the entire Mishnah, in Hebrew and English.

In [7]:
mishnah_path = r'/Users/lee/Judaism/Sefaria_Contest/sefaria_txt/Mishnah/'

directories = [x[0] for x in os.walk(mishnah_path)]
english = [i for i in directories if 'English' in i and not 'Commentary' in i] 
hebrew = [i for i in directories if 'Hebrew' in i and not 'Commentary' in i] 

mishnah_english = ''
mishnah_hebrew = ''

for direct in english:
    mishnah = open(direct + '/clean.txt', 'rb').read().decode(encoding='utf-8')
    mishnah_english += mishnah
    
for direct in hebrew:
    mishnah = open(direct + '/clean.txt', 'rb').read().decode(encoding='utf-8')
    mishnah_hebrew += mishnah

In [3]:
print('Length of Mishnah: {} characters'.format(len(mishnah_hebrew)))

Length of Mishnah: 1164429 characters


In [8]:
vocab = sorted(set(mishnah_hebrew))
print ('{} unique Hebrew characters'.format(len(vocab)))

english_vocab = sorted(set(mishnah_english))
print ('{} unique English characters'.format(len(english_vocab)))

48 unique Hebrew characters
92 unique English characters


In [5]:
print(vocab)

['\n', ' ', "'", '(', ')', ',', '.', ':', 'ְ', 'ֱ', 'ֲ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֻ', 'ּ', 'ׁ', 'ׂ', 'א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'ך', 'כ', 'ל', 'ם', 'מ', 'ן', 'נ', 'ס', 'ע', 'ף', 'פ', 'ץ', 'צ', 'ק', 'ר', 'ש', 'ת']


In [6]:
print(english_vocab)

['\t', '\n', ' ', '!', '"', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '}', '¼', '¾', '–', '‘', '’', '“', '”', '…', '\uf0b7']


## Remove vowels from the Mishnah?

In [67]:
vowels = ['ְ', 'ֱ', 'ֲ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֻ', 'ּ', 'ׁ', 'ׂ']

## Vectorize the text, and create mapping dictionaries.

In [10]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in mishnah_hebrew])

In [8]:
char2idx

{'\n': 0,
 ' ': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '.': 6,
 ':': 7,
 'ְ': 8,
 'ֱ': 9,
 'ֲ': 10,
 'ִ': 11,
 'ֵ': 12,
 'ֶ': 13,
 'ַ': 14,
 'ָ': 15,
 'ֹ': 16,
 'ֻ': 17,
 'ּ': 18,
 'ׁ': 19,
 'ׂ': 20,
 'א': 21,
 'ב': 22,
 'ג': 23,
 'ד': 24,
 'ה': 25,
 'ו': 26,
 'ז': 27,
 'ח': 28,
 'ט': 29,
 'י': 30,
 'ך': 31,
 'כ': 32,
 'ל': 33,
 'ם': 34,
 'מ': 35,
 'ן': 36,
 'נ': 37,
 'ס': 38,
 'ע': 39,
 'ף': 40,
 'פ': 41,
 'ץ': 42,
 'צ': 43,
 'ק': 44,
 'ר': 45,
 'ש': 46,
 'ת': 47}

## Remove vowels from the Mishnah.

In [65]:
vowels = np.arange(8, 21)

In [66]:
vowels

array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

## Split the text up into training sets.

In [11]:
seq_length = 400
examples_per_epoch = len(mishnah_hebrew)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

מ
ֵ
א
ֵ
י


In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [13]:
sequences

<BatchDataset shapes: (401,), types: tf.int64>

In [11]:
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'מֵאֵימָתַי קוֹרִין אֶת שְׁמַע בְּעַרְבִית. מִשָּׁעָה שֶׁהַכֹּהֲנִים נִכְנָסִים לֶאֱכֹל בִּתְרוּמָתָן, עַד סוֹף הָאַשְׁמוּרָה הָרִאשׁוֹנָה, דִּבְרֵי רַבִּי אֱלִיעֶזֶר. וַחֲכָמִים אוֹמְרִים, עַד חֲצוֹת. רַבָּן גַּמְלִיאֵל אוֹמֵר, עַד שֶׁיַּעֲלֶה עַמּוּד הַשָּׁחַר. מַעֲשֶׂה שֶׁבָּאוּ בָנָיו מִבֵּית הַמִּשְׁתֶּה, אָמְרוּ לוֹ, לֹא קָרִינוּ אֶת שְׁמַע. אָמַר לָהֶם, אִם לֹא עָלָה עַמּוּד הַשַּׁחַר, חַיָּב'
'ִין אַתֶּם לִקְרוֹת. וְלֹא זוֹ בִּלְבַד, אֶלָּא כָּל מַה שֶּׁאָמְרוּ חֲכָמִים עַד חֲצוֹת, מִצְוָתָן עַד שֶׁיַּעֲלֶה עַמּוּד הַשָּׁחַר. הֶקְטֵר חֲלָבִים וְאֵבָרִים, מִצְוָתָן עַד שֶׁיַּעֲלֶה עַמּוּד הַשָּׁחַר. וְכָל הַנֶּאֱכָלִים לְיוֹם אֶחָד, מִצְוָתָן עַד שֶׁיַּעֲלֶה עַמּוּד הַשָּׁחַר. אִם כֵּן, לָמָּה אָמְרוּ חֲכָמִים עַד חֲצוֹת, כְּדֵי לְהַרְחִיק אֶת הָאָדָם מִן הָעֲבֵרָה: \nמֵאֵימָתַי קוֹרִין אֶ'
'ת שְׁמַע בְּשַׁחֲרִית. מִשֶּׁיַּכִּיר בֵּין תְּכֵלֶת לְלָבָן. רַבִּי אֱלִיעֶזֶר אוֹמֵר, בֵּין תְּכֵלֶת לְכַרְתִּי. וְגוֹמְרָהּ עַד הָנֵץ הַחַמָּה. רַבִּי יְהוֹשֻׁעַ אוֹמֵר, עַד שָׁלֹשׁ שָׁעו

## Define a function to map from input text to target text (just the input text shifted to the right by one character).

In [15]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'מֵאֵימָתַי קוֹרִין אֶת שְׁמַע בְּעַרְבִית. מִשָּׁעָה שֶׁהַכֹּהֲנִים נִכְנָסִים לֶאֱכֹל בִּתְרוּמָתָן, עַד סוֹף הָאַשְׁמוּרָה הָרִאשׁוֹנָה, דִּבְרֵי רַבִּי אֱלִיעֶזֶר. וַחֲכָמִים אוֹמְרִים, עַד חֲצוֹת. רַבָּן גַּמְלִיאֵל אוֹמֵר, עַד שֶׁיַּעֲלֶה עַמּוּד הַשָּׁחַר. מַעֲשֶׂה שֶׁבָּאוּ בָנָיו מִבֵּית הַמִּשְׁתֶּה, אָמְרוּ לוֹ, לֹא קָרִינוּ אֶת שְׁמַע. אָמַר לָהֶם, אִם לֹא עָלָה עַמּוּד הַשַּׁחַר, חַיָּ'
Target data: 'ֵאֵימָתַי קוֹרִין אֶת שְׁמַע בְּעַרְבִית. מִשָּׁעָה שֶׁהַכֹּהֲנִים נִכְנָסִים לֶאֱכֹל בִּתְרוּמָתָן, עַד סוֹף הָאַשְׁמוּרָה הָרִאשׁוֹנָה, דִּבְרֵי רַבִּי אֱלִיעֶזֶר. וַחֲכָמִים אוֹמְרִים, עַד חֲצוֹת. רַבָּן גַּמְלִיאֵל אוֹמֵר, עַד שֶׁיַּעֲלֶה עַמּוּד הַשָּׁחַר. מַעֲשֶׂה שֶׁבָּאוּ בָנָיו מִבֵּית הַמִּשְׁתֶּה, אָמְרוּ לוֹ, לֹא קָרִינוּ אֶת שְׁמַע. אָמַר לָהֶם, אִם לֹא עָלָה עַמּוּד הַשַּׁחַר, חַיָּב'


In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 35 ('מ')
  expected output: 12 ('ֵ')
Step    1
  input: 12 ('ֵ')
  expected output: 21 ('א')
Step    2
  input: 21 ('א')
  expected output: 12 ('ֵ')
Step    3
  input: 12 ('ֵ')
  expected output: 30 ('י')
Step    4
  input: 30 ('י')
  expected output: 35 ('מ')


## Create training batches.

In [16]:
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 400), (64, 400)), types: (tf.int64, tf.int64)>

## Build the RNN model.

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                 batch_input_shape=[batch_size, None]),
                                 tf.keras.layers.GRU(rnn_units,
                                                     return_sequences=True,
                                                     stateful=True,
                                                     recurrent_initializer='glorot_uniform'),
                                 tf.keras.layers.Dense(vocab_size)])
    return model

In [19]:
model = build_model(vocab_size = len(vocab),
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units,
                    batch_size=BATCH_SIZE)

In [19]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           12288     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 48)            49200     
Total params: 3,999,792
Trainable params: 3,999,792
Non-trainable params: 0
_________________________________________________________________


## Predict characters with untrained RNN.

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 400, 48) # (batch_size, sequence_length, vocab_size)


In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
sampled_indices

array([ 4,  8,  7, 47, 42, 16, 15,  2, 11, 46, 17, 23, 31,  3, 23, 14,  5,
       31, 30, 22, 21, 27, 36, 11, 37, 42,  9,  2, 19,  4, 24, 44,  1,  5,
       31, 41,  3, 12, 41, 38, 21, 34, 30, 38, 39, 11, 30, 26,  3, 24, 11,
        1, 43, 11,  7,  0, 46, 19, 10, 15, 34,  4, 23,  2, 34, 47, 32,  4,
       37, 11, 21,  7, 10,  9, 13, 36, 47, 17, 37, 15, 46, 43, 23, 15, 41,
       10, 11, 41, 26, 31, 11, 16,  0, 30, 15, 45, 18, 36,  2, 24,  4, 41,
       36, 13,  5, 25, 28, 44,  2, 14, 29, 32, 10, 27,  4, 34, 36, 35, 21,
       34, 14, 36, 46,  4, 25, 13, 19, 17, 47, 30, 22, 43, 16, 10, 17,  7,
       31, 40, 29, 13, 15,  6,  9, 41, 32, 27, 29,  9, 10, 28,  7, 17, 10,
        4, 44, 36,  3, 45, 10, 22,  5,  1, 35,  4, 14, 34, 45,  6, 14, 14,
        0, 36,  1,  3, 47, 39, 18, 37, 27,  9, 34, 37, 29, 26,  5, 16, 19,
        3, 18, 39, 31, 36, 34,  4, 30, 38, 34, 25, 14,  9, 40,  8, 41, 14,
        4, 40, 40, 46, 12, 40, 14, 43, 37, 34, 41, 40, 11, 10,  1, 32, 20,
       15, 44,  8, 46, 30

In [26]:
print('Input: \n', repr(''.join(idx2char[input_example_batch[0]])))
print()
print('Predictions: \n', repr(''.join(idx2char[sampled_indices])))

Input: 
 'ם לַעֲמֹד בְּשָׁלֹשׁ שָׁעוֹת. הַקּוֹרֵא מִכָּאן וְאֵילָךְ לֹא הִפְסִיד, כְּאָדָם הַקּוֹרֵא בַתּוֹרָה: \nבֵּית שַׁמַּאי אוֹמְרִים, בָּעֶרֶב כָּל אָדָם יַטּוּ וְיִקְרְאוּ, וּבַבֹּקֶר יַעַמְדוּ, שֶׁנֶּאֱמ'

Predictions: 
 "ֻזְֱִׂ\nרם'ְֲ\nט:דֻׂאחוו.חנ:'שזתלת,ֵמ,(חֹונףיָ(םהּׂ(םעןך(ס'שָאףךכ\nֱפׁ\nֶׁ.צרע'מַּׁאצטיכץעת,,ד.ַָ.ֲֲט)ְלַש ףְ.ְש,לְצָ..ָ ְםמףךאַבהמנכ(ֲטרֻץגכִ,ֻףֱז)ץצלֵ:ׁמץ.טְֵ,ף:ׁ\n\nֱזפה'זרּף.ָקאל.ב דקט:טשלּלמֲָ,נג:ֹץחוג"


## Train the model.

In [23]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 400, 48)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       3.8713052


In [24]:
model.compile(optimizer='adam', loss=loss)

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS=20 #30

In [26]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
45/45 [==============================] - 157s 3s/step - loss: 3.2965
Epoch 2/20
45/45 [==============================] - 159s 4s/step - loss: 2.0513
Epoch 3/20
45/45 [==============================] - 159s 4s/step - loss: 1.6395
Epoch 4/20
45/45 [==============================] - 160s 4s/step - loss: 1.2376
Epoch 5/20
45/45 [==============================] - 160s 4s/step - loss: 0.7820
Epoch 6/20
45/45 [==============================] - 160s 4s/step - loss: 0.3774
Epoch 7/20
45/45 [==============================] - 160s 4s/step - loss: 0.1726
Epoch 8/20
45/45 [==============================] - 172s 4s/step - loss: 0.1146
Epoch 9/20
45/45 [==============================] - 178s 4s/step - loss: 0.0949
Epoch 10/20
45/45 [==============================] - 183s 4s/step - loss: 0.0853
Epoch 11/20
45/45 [==============================] - 184s 4s/step - loss: 0.0777
Epoch 12/20
45/45 [==============================] - 185s 4s/step - loss: 0.0733
Epoch 13/20
45/45 [==================

In [21]:
def generate_text(model, start_string, num_generate=2000, temperature=1.0):
    # Evaluation step (generating text using the learned model)
    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    # Empty string to store our results
    text_generated = []
    
    # Here batch size == 1
    model.reset_states()
    for i in np.arange(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
        
    return start_string + ''.join(text_generated)

In [22]:
fmodel = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
fmodel.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
fmodel.build(tf.TensorShape([1, None]))

In [23]:
print(generate_text(fmodel, u'מֵאֵימָתַי'))

מֵאֵימָתַי קוֹרִין אֶת שְׁמַע בְּשַׁחֲרִית. מִשֶּׁיַּכִּיר בֵּין תְּכֵלֶת לְלָבָן. רַבִּי אֱלִיעֶזֶר אוֹמֵר, בָּהֶן חַיָּבִין. אֵלּוּ וָאֵלּוּ פְּטִיָּה אִם שָׁמֹעַ לְוַיֹּאמֶר, בֵּין וַיֹּאמֶר לֶאֱמֶת וְיַצִּיב. רַבִּי יְהוּדָה אוֹמֵר, בֵּין וַיֹּאמֶר לֶאֱמֶת וְיַצִּיב לֹא יַפְסִיק. אָמַר רַבִּי יְהוֹשֻׁעַ בֶּן קָרְחָה, לָמָּה קָדְמָה שְׁמַע לִוְהָיָה אִם שָׁמֹעַ, אֶלָּא כְדֵי שֶׁיְּקַבֵּל עָלָיו עֹל מַלְכוּת שָׁמַיִם תְּחִלָּה, וְאַחַר כָּךְ יְקַבֵּל עָלָיו עֹל מִצְוֹת. וְהָיָהָא הַשָּׁחַר. הֶקְטֵר חֲלָבִים וְאֵבָרִים, מִצְוָתָן עַד שֶׁיַּעֲלֶה עַמּוּד הַשָּׁחַר. אִם כֵּן, לָמָּה אָמְרוּ חֲכָמִים עַד חֲצוֹת, כְּדֵי לְהַרְחִיק אֶת הָאָדָם מִן הָעֲבֵרָה: 
מֵאֵימָתַי קוֹרִין אֶת שְׁמַע בֶּשְׁתֵּיוּ וַחֲכָלִים הַיּוֹם וְאַחַר כָּךְ מְבָרֵךְ עַל הַיַּיִן. וּבֵית הִלֵּל אוֹמְרִים, מְבָרֵךְ עַל הַיַּיִן 
חַיָּב אָדָם לְבָרֵךְ לְאַחֲרָיו, וְאֵינוֹ מְבָרֵךְ לְפָנָיו. רַבִּי יְהוּדָה אוֹמֵר, מְבָרֵךְ לִפְנֵיהֶם וּלְאַחֲרֵיהֶם: 
הָיָה עוֹמֵד בַּתְּפִלָּה, וְנִזְכַּר שֶׁהוּא בַעַל קְרִי, לֹא יַפ

In [24]:
print(generate_text(fmodel, u'בְּ'))

בְּרָבָּה וְאַחַר כָּךְ מְבָרֵךְ עַל הַיַּיִן. וּבֵית הִלֵּל אוֹמְרִים, מְבָרֵךְ עַל הַיַּיִן וְאַחַר כָּךְ מְבָרֵךְ עַל הַיּוֹם: 
בֵּית שַׁמַּאי אוֹמְרִים, נִפְנָס יִטֹּם: 
מִי שֶׁמֵּתוֹ וּלְאַחֲרֵיהֶם שָׁלשׁ בְּרָכוֹת, דִּבְרֵי רַבָּן גַּמְלִיאֵל. וַחֲכָמִים אוֹמְרִים, בְּרָכָה אַחַת מֵעֵין שָׁלשׁ. רַבִּי עֲקִיבָא אוֹמֵר, אֲפִלּוּ אָכַל שֶׁלֶק וְהוּא מְזוֹנוֹ, מְבָרֵךְ אַחֲרָיו שָׁלשׁ בְּרָכוֹת. הַשּׁוֹתֶה מַיִם לִצְמָאוֹ, אוֹמֵר שֶׁהַכֹּל נִהְיֶה בִּדְבָרוֹ. רַבִּי טַרְפוֹן אוֹמֵר, בּוֹרֵא נְפָשׁוֹת רַבּוֹת: 
שְׁלשָׁה שֶׁאָכְלוּ כְאֶחָד, חַיָּבִין לְזַמֵּן. אָכַל דְּמַאי, וּמַעֲשֵׂר רִאשׁוֹן שֶׁנִּטְּלָה תְרוּמָתוֹ, וּמַעֲשֵׂר שֵׁנִי וְהֶקְדֵּשׁ חֶלֶי עֲקוֹדָה זָרָה. אֵין מְבָרְכִין עַל הַנֵּר עַד שֶׁיֵּאוֹתוּ לְאוֹרוֹ: 
מִי שֶׁאָכַל וְשָׁכַח וְלֹא בֵרַךְ, בֵּית שַׁמַּאי אוֹמְרִים, יַחֲזֹר לִמְקוֹמוֹ וִיבָרֵךְ. וּבֵית הִלֵּל אוֹמְרִים, יְבָרֵךְ בַּמָּקוֹם שֶׁנִּזְכָּר. עַד אֵימָתַי הוּא מְבָרֵךְ. עַד כְּדֵי שֶׁיִּתְעַכֵּל הַמָּזוֹן שֶׁבְּמֵעָיו: 
בָּא לָהֶם יַיִן לְאַחַר הַמָּזוֹן ו

In [25]:
print(generate_text(fmodel, u'עַל'))

עַל הַבְּשָׂמִים שֶׁלִּפְנֵי עֲבוֹדָה זָרָה. אֵין מְבָרְכִין עָלָיו: 
הָיוּ לְפָנָיו מִינִים הַרְבֵּה, רַבֵּנוּ, שֶׁאָבֵל אָסוּר לִרְחֹץ. אָמַר לָהֶם, אֵינִי כִשְׁאָר כָּל אָדָם, אִססטְרֵא לְמַפְרֵעַ, לֹא יָצָא. קָרָא וְטָעָה, יַחֲזֹר לְמָקוֹם שֶׁטָּעָה: 
הָאֻמָּנִין קוֹרִין בְּרֹאשׁ הָאִילָן אוֹ בְּרוֹבָן אַי רָעו, לֹא יַפְסִיק: 
מַזְכִּירִין גְּבוּרוֹת גְּשָׁמִים בִּתְחִיַּת הַמֵּתִים, וְנוֹתֵן הוֹדָאָה: 
הָאוֹמֵר עַל קַן צִפּוֹר יַגִּיעוּ רַחֲמֶיךָ, וְעַל טוֹב יִזַּכּוּר הַרִּיבָן אִם רָקוֹת אוֹמֵר בָּרוּךְ דַּיַּן הָאֱמֶת: 
בָּנָה בַיִת חָדָשׁ, וְקָנָה כֵלִים חֲדָשִׁים, אוֹמֵר בָּרוּזֶה וְטוּ אֱלֹהִים, יְיָ מִמְּקוֹר יִשְׂרָאֵר (תהלים סח). אָמַר רַבִּי עֲקִיבָא, מַה מָּצְאָדָם נוֹ. כָּל אָדָם קוֹרֵא כְדַרְכּוֹ, שֶׁנֶּאֱמַר (שם) וּבְלֶכְתְּךָ בַדֶּרֶךְ. אִם כֵּן, לָמָּה נֶאֱמַר וּבְשָׁכְבְּךָ וּבְקוּמֶךָ, בְּשָׁעָה שֶׁבְּנֵי אָדָם שׁוֹכְבִים, וּבְשָׁעָה שֶׁבְּנֵי אָדָם עוֹמְדִים. אָמַר רַבִּי טַרְפוֹן, אֲנִי הָיִיהִי רָצוֹן שֶׁתֵּלֵד אִשְׁתִּי זָכָר, הֲרֵי זוֹ תְּפִלַּת שָׁוְא. הָיָ